In [1]:
import os

In [2]:
# settings

n = '400' # default: 400, less for testing purposes : e.g. 50
year = "" # unused
channel = "total" # options are mu, ele and total
spinpostfix = "_Spin32" # options are "" and "_Spin32"
region = "SR"

injected_signal = 0.2
rmin = -10
rmax = 10
masspoint = "1200"

datacard = "cards_combined/" + masspoint + "_pt_ST_" + channel + "_" + region + spinpostfix + ".dat"

workdir = "workdir_signal_injection"

do_condor = True
path = "/nfs/dust/cms/user/flabe/TstarTstar/combine/CMSSW_10_2_13/src/HiggsAnalysis/CombinedLimit/data/TstarTstar/"+workdir

tag = region + "_M-" + masspoint + spinpostfix + "_" + year + "_" + channel + "_" + str(injected_signal)
print(tag)

bash = "#!/bin/bash\n"

SR_M-1200_Spin32__total_0.2


In [3]:
# add environment setting if condor
if do_condor:
    bash += 'source /cvmfs/cms.cern.ch/cmsset_default.sh\n'
    bash += 'cd ~\n'
    bash += 'cd /nfs/dust/cms/user/flabe/TstarTstar/combine/CMSSW_10_2_13/src\n'
    bash += 'eval `scramv1 runtime -sh`'
    bash += 'cd ' + path + '\n'

In [4]:
# step 0: first lets create a working directory
if (os.path.isdir(workdir)): print("workdir already exists")
else: os.mkdir(workdir)

# first command to be run is text2workspace
text2workspacecommand = "text2workspace.py ../{} -o workspace.root".format(datacard, workdir)
print(text2workspacecommand)
bash += text2workspacecommand + "\n"
bash += 'echo "Done text2workspace"\n'

workdir already exists
text2workspace.py ../cards_combined/1200_pt_ST_total_SR_Spin32.dat -o workspace.root


In [5]:
# step 1: initial fit

command_step1 = "combine -M MultiDimFit" # command base
command_step1 += ' -d workspace.root' # adding workspace
command_step1 += ' -v 1' # print out verbosity level
command_step1 += ' -m 125' # higgs mass, because why not
command_step1 += ' -t {}'.format(n) # toys
command_step1 += ' --setParameters r={}'.format(injected_signal) # signal to be injected
command_step1 += ' --rMin {} --rMax {}'.format(rmin, rmax) # some r ranges
command_step1 += ' --saveWorkspace' # saving the resulting workspace?
command_step1 += ' --algo=singles' # ???
command_step1 += ' -n {}'.format(tag) # tag
command_step1 += ' --cminDefaultMinimizerStrategy 0'
command_step1 += ' --robustFit=1 --robustHesse 1'
command_step1 += ' > step1_{}.log'.format(tag) # log

print(command_step1)
bash += command_step1 + "\n"
bash += 'echo "Done step 1"\n'

combine -M MultiDimFit -d workspace.root -v 1 -m 125 -t 400 --setParameters r=0.2 --rMin -10 --rMax 10 --saveWorkspace --algo=singles -n SR_M-1200_Spin32__total_0.2 --cminDefaultMinimizerStrategy 0 --robustFit=1 --robustHesse 1 > step1_SR_M-1200_Spin32__total_0.2.log


In [6]:
# step 2: generating toys

command_step2 = 'combine -M GenerateOnly' # command base
command_step2 += ' -d higgsCombine{}.MultiDimFit.mH125.123456.root'.format(tag) # workspace from step1
command_step2 += ' -m 125' # higgs mass, because why not
command_step2 += ' --expectSignal {}'.format(injected_signal)
command_step2 += ' --rMin {} --rMax {}'.format(rmin, rmax) # some r ranges
command_step2 += ' -t {} --toysFrequentist --bypassFrequentistFit'.format(n) # toys
command_step2 += ' --saveToys'
command_step2 += ' --snapshotName "MultiDimFit"'
command_step2 += ' -n {}'.format(tag) # tag
command_step2 += ' >> step2_{}.log'.format(tag) # log

print(command_step2)
bash += command_step2 + "\n"
bash += 'echo "Done step 2"\n'

combine -M GenerateOnly -d higgsCombineSR_M-1200_Spin32__total_0.2.MultiDimFit.mH125.123456.root -m 125 --expectSignal 0.2 --rMin -10 --rMax 10 -t 400 --toysFrequentist --bypassFrequentistFit --saveToys --snapshotName "MultiDimFit" -n SR_M-1200_Spin32__total_0.2 >> step2_SR_M-1200_Spin32__total_0.2.log


In [7]:
# step 3: running the fit

command_step3 = 'combine -M FitDiagnostics'
command_step3 += ' -d higgsCombine{}.MultiDimFit.mH125.123456.root'.format(tag) # workspace from step1
command_step3 += ' --toysFile higgsCombine{}.GenerateOnly.mH125.123456.root'.format(tag) # toys file from step 2
command_step3 += ' -t {} --toysFrequentist --bypassFrequentistFit'.format(n) 
command_step3 += ' --setParameters r={}'.format(injected_signal)
command_step3 += ' --rMin {} --rMax {}'.format(rmin, rmax) # some r ranges
command_step3 += ' --saveWorkspace'
command_step3 += ' --noErrors --minos none'
command_step3 += ' -n {}'.format(tag) # tag
command_step3 += ' --cminDefaultMinimizerStrategy 0'
command_step3 += ' --robustFit=1 --robustHesse 1'
command_step3 += ' >> step3_{}.log'.format(tag) # log

print(command_step3)
bash += command_step3 + "\n"
bash += 'echo "Done step 3"\n'

combine -M FitDiagnostics -d higgsCombineSR_M-1200_Spin32__total_0.2.MultiDimFit.mH125.123456.root --toysFile higgsCombineSR_M-1200_Spin32__total_0.2.GenerateOnly.mH125.123456.root -t 400 --toysFrequentist --bypassFrequentistFit --setParameters r=0.2 --rMin -10 --rMax 10 --saveWorkspace --noErrors --minos none -n SR_M-1200_Spin32__total_0.2 --cminDefaultMinimizerStrategy 0 --robustFit=1 --robustHesse 1 >> step3_SR_M-1200_Spin32__total_0.2.log


In [8]:
# step 4: plotting

command_step4 = 'python ../plot_bias_pull.py'
command_step4 += ' -i fitDiagnostics{}.root'.format(tag) # input file
command_step4 += ' -t {}'.format(n) # toys
command_step4 += ' -r {}'.format(injected_signal) # injected signal
command_step4 += ' -o {}'.format(tag) # tag
command_step4 += ' --label "{}, {}NEWLINEm(t*) = {}"'.format(year if year != "" else "full run 2", "ele + mu" if channel == "total" else channel, masspoint + spinpostfix.replace("_Spin32", "spin 3/2")) # tag
command_step4 += ' >> step4_{}.log'.format(tag) # log

print(command_step4)
bash += command_step4 + "\n"
bash += 'echo "Done step 4"\n'

python ../plot_bias_pull.py -i fitDiagnosticsSR_M-1200_Spin32__total_0.2.root -t 400 -r 0.2 -o SR_M-1200_Spin32__total_0.2 --label "full run 2, ele + muNEWLINEm(t*) = 1200spin 3/2" >> step4_SR_M-1200_Spin32__total_0.2.log


In [9]:
# put all into a bash file
with open(workdir+"/signal_injection_script.sh", "w") as file:
    file.write(bash)

In [10]:
# create a condor submit script for that bash script

script = ""

script += 'executable = '+path+'/signal_injection_script.sh\n'
script += 'output = '+path+'/condor.log\n'
script += 'error = '+path+'/condor.log\n'
script += 'batch_name = signal_injection_test\n'
script += 'queue 1'

with open(workdir+"/condor.submit", "w") as file:
    file.write(script)